In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd


In [ ]:
from app.flipside.models import Flipside
from app.oracles.models import Oracle
from app.constants import (
    oracle_feeds_lookup, oracle_feeds, 
    phase_windows, timestamp_windows,
    filenames_filtered
    )
from app.aave.models2 import AaveBalancePlotter
from app.combo.models import Combo
from app.swaps.models import DexSwaps
from app.visuals.plotly_helper import combined_action_layout, combined_action_layout_time

In [ ]:
fs = Flipside(filenames_filtered)

In [ ]:
oracles = Oracle(fs, oracle_feeds_lookup)

In [ ]:
swaps = DexSwaps(fs)

In [ ]:
volume_hourly = pd.DataFrame(swaps.hourly)
volume_daily = pd.DataFrame(swaps.daily)

In [ ]:
aave = AaveBalancePlotter(fs)
aave.iterate_through_blocks()

# Vars

In [ ]:
target_addresses = ['0x57e04786e231af3343562c062e0d058f25dace9e', '0x7a16ff8270133f063aab6c9977183d9e72835428']
window_start = 14710000 - 420
window_end = 16027236 + 420

title = "Avi"

# Process Vars

In [ ]:
data = []
for target_address in target_addresses:
    combo = Combo(aave, oracles, target_address)
    combo.process_value(window_start, window_end)
    combo.process_sums()
    df_combo_crv = pd.DataFrame(combo.records['CRV'])

    df_aave_sums = pd.DataFrame(combo.sum_records)
    data.append({'crv': df_combo_crv,
                'sum': df_aave_sums})


In [ ]:
# df_combo_crv = pd.DataFrame(combo.records['CRV'])

# df_aave_sums = pd.DataFrame(combo.sum_records)

# Bullets

In [ ]:
def make_bullet(max_ltv, debt_usd, collateral_usd, val_name='value', height = 250):

    # max_ltv = 0.9
    # debt_usd = 1700
    # collateral_usd = 2000

    buffer = debt_usd * (1/max_ltv)
    ltv_value = collateral_usd * max_ltv
    position_value = collateral_usd - debt_usd

    padding = collateral_usd - debt_usd - buffer


    fig = go.Figure(go.Indicator(
        mode = "number+gauge+delta", value = position_value,
        domain = {'x': [0, 1], 'y': [0, 1]},
        title = {'text' :"<b>Value</b>"},
        # How
        delta = {'reference': (collateral_usd * max_ltv) - debt_usd},

        gauge = {
            'shape': "bullet",

            'axis': {'range': [None, collateral_usd]},

            'threshold': {
                'line': {'color': "red", 'width': 2},
                'thickness': 0.75,
                'value': ltv_value},
            'steps': [
                {'range': [0, debt_usd], 'color': "tomato", 'name': 'Test'},
                {'range': [debt_usd, collateral_usd], 'color': "grey"}]}))

    fig.update_layout(height = height,
                    width=800)
    fig.update_layout(
        title=f"Collateral to value: {title}",
    #     xaxis_title="X Axis Title",
    #     yaxis_title="Y Axis Title",
    #     legend_title="Legend Title",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    return fig

    

In [ ]:
1 / (4+1)


In [ ]:
def make_multi_bullet(fig, title, max_ltv, debt_usd, collateral_usd, expected_total, count=0):
    # print(debt_usd)
    # print(collateral_usd)
    split_size = 1 /((expected_total * 1.5) + 1) 
    padding = split_size 
    start = (split_size * count) + padding *count
    end = split_size * (count + 1) + padding *count
    print("___")
    print(start)
    print(end)
    fig.add_trace(
        go.Indicator(
        mode = "number+gauge+delta", 
        value = collateral_usd - debt_usd,
        delta = {'reference': (collateral_usd-debt_usd) - (collateral_usd * max_ltv - debt_usd) },
        domain = {'x': [0.2, 1], 'y': [start, end]},
        title = {'text': title},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 150000000]},
            'threshold': {
                'line': {'color': "red", 'width': 2},
                'thickness': 0.75,
                'value': collateral_usd * max_ltv},
            'steps': [
                {'range': [0, debt_usd], 'color': "red"},
                {'range': [debt_usd, collateral_usd], 'color': "lightgray"}],
            'bar': {'color': "black"}
            }))
    return fig

In [ ]:
# target_block_start = timestamp_windows['michael_ramp_up']
# target_block_end = timestamp_windows['michael_ramp_up']

target_block_start = 15900000
target_block_end = 15935500

df_aave_sums_0 = data[0]['sum']
df_aave_sums_1 = data[1]['sum']

phase = "Michael Ramp Up"

fig = go.Figure()

df_filtered = df_aave_sums_0[df_aave_sums_0['block_number'] <= target_block_start-10]
debt_usd_0 = float(df_filtered.tail(1)['d_sum_usd_value'])
collateral_usd_0 = float(df_filtered.tail(1)['c_sum_usd_value'])
max_ltv_0 = 0.88
fig = make_multi_bullet(fig, "Avi Start", max_ltv_0, debt_usd_0, collateral_usd_0, 4, 1)

df_filtered = df_aave_sums_0[df_aave_sums_0['block_number'] <= target_block_end+10]
debt_usd_0 = float(df_filtered.tail(1)['d_sum_usd_value'])
collateral_usd_0 = float(df_filtered.tail(1)['c_sum_usd_value'])
fig = make_multi_bullet(fig, "Avi End", max_ltv_0, debt_usd_0, collateral_usd_0, 4, 0)

df_filtered = df_aave_sums_1[df_aave_sums_1['block_number'] <= target_block_start -10]
debt_usd_1 = float(df_filtered.tail(1)['d_sum_usd_value'])
collateral_usd_1 = float(df_filtered.tail(1)['c_sum_usd_value'])
max_ltv_1 = 0.65
fig = make_multi_bullet(fig, "Michael Start", max_ltv_1, debt_usd_1, collateral_usd_1, 4, 3)

df_filtered = df_aave_sums_1[df_aave_sums_1['block_number'] <= target_block_end+10]
debt_usd_1 = float(df_filtered.tail(1)['d_sum_usd_value'])
collateral_usd_1 = float(df_filtered.tail(1)['c_sum_usd_value'])
fig = make_multi_bullet(fig, "Michael End", max_ltv_1, debt_usd_1, collateral_usd_1, 4, 2)

fig.update_layout(
    title=f"Aave Positions Sumarized: {phase}",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()


In [ ]:
target_time = timestamp_windows['michael_ramp_up']


In [ ]:
make_bullet(0.88, debt_usd_0, collateral_usd_0, 'Avi')
# make_bullet(0.65, debt_usd_1, collateral_usd_1, 'Michael')

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Indicator(
    mode = "number+gauge+delta", value = collateral_usd_0 - debt_usd_0,
    delta = {'reference': (collateral_usd_0 * max_ltv_0) - debt_usd_0},
    domain = {'x': [0.1, 1], 'y': [0.2, 0.4]},
    title = {'text': "Avi"},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None, 150000000]},
        'threshold': {
            'line': {'color': "red", 'width': 2},
            'thickness': 0.75,
            'value': collateral_usd_0 * max_ltv_0},
        'steps': [
            {'range': [0, debt_usd_0], 'color': "red"},
            {'range': [debt_usd_0, collateral_usd_0], 'color': "lightgray"}],
        'bar': {'color': "black"}}))

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta", value = collateral_usd_1 - debt_usd_1,
    delta = {'reference': (collateral_usd_1 * max_ltv_1) - debt_usd_1},
    domain = {'x': [0.1, 1], 'y': [0.6, 0.8]},
    title = {'text': "Michael"},
    gauge = {
        'shape': "bullet",
        'axis': {'range': [None, 150000000]},
        'threshold': {
            'line': {'color': "red", 'width': 2},
            'thickness': 0.75,
            'value': collateral_usd_1 * max_ltv_1},
        'steps': [
            {'range': [0, debt_usd_1], 'color': "red"},
            {'range': [debt_usd_1, collateral_usd_1], 'color': "lightgray"}],
        'bar': {'color': "black"}}))

# fig.add_trace(go.Indicator(
#     mode = "number+gauge+delta", value = 220,
#     delta = {'reference': 200},
#     domain = {'x': [0.25, 1], 'y': [0.7, 0.9]},
#     title = {'text' :"Satisfaction"},
#     gauge = {
#         'shape': "bullet",
#         'axis': {'range': [None, 300]},
#         'threshold': {
#             'line': {'color': "black", 'width': 2},
#             'thickness': 0.75,
#             'value': 210},
#         'steps': [
#             {'range': [0, 150], 'color': "gray"},
#             {'range': [150, 250], 'color': "lightgray"}],
#         'bar': {'color': "black"}}))
# fig.update_layout(height=300, margin = {'t':0, 'b':0, 'l':0})
fig.update_layout(
    title=f"Aave Positions Sumarized: {phase}",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ),
    # height=300
)

fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig = go.Figure()
fig.update_layout(
    title=f"Aave Total Position Values: {title} (USD)",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig = fig.add_trace(
    go.Scatter(
        x=df_filtered.block_number,
        y=df_filtered.ltv,
        name="LTV",
        marker_color='purple'
    ),
    secondary_y=True
)

fig = fig.add_trace(
    go.Scatter(
        x = df_filtered.block_number,
        y = df_filtered.c_sum_usd_value, 
        name = "Collateral USD Value",
        mode='lines',
        marker_color="green",
    ),
    secondary_y=False
)


fig = fig.add_trace(
    go.Scatter(
        x = df_filtered.block_number,
        y = df_filtered.d_sum_usd_value, 
        name = "Debt USD Value",
        mode='lines',
        marker_color="red"
    ),
    secondary_y=False
)



# 
fig.add_vrect(x0=timestamp_windows['avi_initial_deposit'], x1=timestamp_windows['avi_attack_start'], 
            annotation_text='Avi Setup', annotation_position="top left",
            fillcolor='orange', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_attack_start'], x1=timestamp_windows['avi_initial_liquidation'], 
            annotation_text='Attack', annotation_position="top left",
            fillcolor='blue', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)



fig.add_vrect(x0=timestamp_windows['michael_ramp_up'], x1=timestamp_windows['michael_ramp_stop'], 
            annotation_text='Michael Movements', annotation_position="top left",
            fillcolor='blue', opacity=0.25, line_width=0)

# fig.update_yaxes(range=[0, 140000000])

# fig.update_yaxes(range=[0, 50000000])

fig.add_hline(y=0.65, line_dash="dot",
            #   annotation_text="Liquidation Threshold",
            #   annotation_position="top left",
              secondary_y=True)

fig.update_yaxes(range=[0, 1], secondary_y=True)


fig.show()